# Generate interval best execution benchmarks using dedicated service - C#

### Overview
Best execution aims to measure the performance of executed trades, an execution algorithm or an execution venue.

This sample is based on an `interval` approach and designed to illustrate market activity over a look back period.

<div class="alert alert-block alert-info">
    <b>Note:</b> To explore the point-in-time approach, suitable to highlight individual tradres among market activity. Please refer to <i>bestex-pointintime</i> notebooks.
</div>

This sample demonstrates how to request and plot from a dedicated data service **on-demand** best execution *interval* benchmarks.

### Inputs/outputs
Best execution interval sample requires instrument's identifier, date time intervals as per inputs. Results are as follows:
* a snapshot of occured trades
* a set of computed benchmarks based on the occured trades

### Services used
This sample uses *gRPC requests* in order to retrieve trades information from the hosted service. The queried endpoint in this script are:
* *TickTradesService*: to directly retrieve trades data from the server.

### Packages required
1. Systemathics:
    * *Systemathics.Apis.Type.Shared.V1*
    * *Systemathics.Apis.Services.Tick.V1*
2. Open source:
    * *Google.Protobuf.WellKnownTypes*
    * *Google.Type*
    * *Grpc.Net.Client*
    * *Grpc.Core*
    * *XPlot.Plotly* as per display package
    
***

# Run BestEx interval sample

### Step 1: Install packages

In [ ]:
#r "nuget: Systemathics.Apis, 0.9.27"
#r "nuget: XPlot.Plotly.Interactive"
#r "nuget: CsvHelper, 22.1.2"

In [ ]:
using Systemathics.Apis.Type.Shared.V1;
using Systemathics.Apis.Services.Tick.V1;
using Google.Protobuf.WellKnownTypes;
using Google.Type;
using Grpc.Net.Client;
using Grpc.Core;
// Plot charts
using XPlot.Plotly; 
// Import/export csv files 
using CsvHelper;
using System.IO;
using System.Globalization;

### Step 2: Prepare API requests
The following code snippets retrieve authentication token and prepare the API request by: opening the *channel* and adding the *token* to the request header:

In [ ]:
// Retrieve authentication token
var token = System.Environment.GetEnvironmentVariable("AUTH0_TOKEN");

// Add token to the request header
var headers = new Metadata();
headers.Add("Authorization", $"Bearer {token}");

// Define API entry
var channel = GrpcChannel.ForAddress("https://apis.systemathics.cloud");

In [ ]:
// Display authentication token 
display(token);

### Step 3: Create and process request
To request *tick data* service, we need to specify:
* Instrument identifier
* Time period selection: select start and end dates
* Tick data request parameters

#### 3.1 Instrument selection

In [ ]:
var ticker = "AAPL";
var exchange = "BATS";

#### 3.2 Time period delimitation

In [ ]:
// Create the date interval (we are using Google date time format)
var day = new DateTime(2021, 03, 15);
var dateIntervals = new DateInterval()
{
    StartDate = new Date { Year = 2021, Month = 03, Day = 15 },
    EndDate = new Date { Year = 2021, Month = 03, Day = 15 }
};

// Create the time interval (we are using Google date time format)
// UTC time zone
var timeInterval = new TimeInterval()
{
    StartTime = new TimeOfDay { Hours = 16, Minutes = 20, Seconds = 00 },
    EndTime = new TimeOfDay { Hours = 16, Minutes = 30, Seconds = 00 } 
};

#### 3.3 Request creation
The following code snippet creates *gRPC client*, process request and returns request reply.

In [ ]:
// Generate constraints based on the previous time selection:
var constraints = new Constraints(); 
constraints.DateIntervals.Add(dateIntervals);
constraints.TimeIntervals.Add(timeInterval);

The following code snippets create request and instantiate the service:

In [ ]:
// Generate the tick trades request
var request = new TickTradesRequest { Constraints = constraints };
var identifier = new Identifier { Exchange = exchange, Ticker = ticker};
request.Identifiers.Add(new [] {identifier}); 

In [ ]:
// Instantiate the tick trades service
var service = new TickTradesService.TickTradesServiceClient(channel);

### Step 4: Visualize data

#### 4.1 Retrieve trade prices and volumes

In [ ]:
// Get the trades
var trades = new List<Trade>();

var call = service.TickTrades(request, headers);
await foreach (var trade in call.ResponseStream.ReadAllAsync())
{
    if(trade.Trade != null)
    {
        trades.Add(trade.Trade);   
    }
}

The following code snippet displays request results:

In [ ]:
display(trades);

#### 4.2 Retrieve interval benchmarks

In [ ]:
// Get bestex time interval
var intervalStart = new DateTime(dateIntervals.StartDate.Year, dateIntervals.StartDate.Month, dateIntervals.StartDate.Day, timeInterval.StartTime.Hours, timeInterval.StartTime.Minutes, timeInterval.StartTime.Seconds);
var intervalEnd = new DateTime(dateIntervals.EndDate.Year, dateIntervals.EndDate.Month, dateIntervals.EndDate.Day, timeInterval.EndTime.Hours, timeInterval.EndTime.Minutes, timeInterval.EndTime.Seconds);

// Get first and last trades' timestamps
var firstTimestamp = trades.Select(elt => elt.TimeStamp.ToDateTime()).First();
var lastTimestamp = trades.Select(elt => elt.TimeStamp.ToDateTime()).Last();

// Get a set of benchmarks
var totalTrades = trades.Count;
var totalSizes = trades.Sum(trade => trade.Size);
var avgPrice = trades.Select(elt => elt.Price).Average();
var avgSize = trades.Select(elt => elt.Size).Average();
var maxPrice = trades.Max(trade => trade.Price);
var minPrice = trades.Min(trade => trade.Price);
var maxSize = trades.Max(trade => trade.Size);
var minSize = trades.Min(trade => trade.Size);
var vwap = trades.Sum(trade => trade.Price * trade.Size ) / totalSizes;

In [ ]:
// Display results
Console.WriteLine($"Interval start time \t: {intervalStart:yyyy-MM-dd HH:mm:ss}");
Console.WriteLine($"Interval end time\t: {intervalEnd:yyyy-MM-dd HH:mm:ss}");
Console.WriteLine($"First trade occured at\t: {firstTimestamp:yyyy-MM-dd HH:mm:ss}");
Console.WriteLine($"Last trade occured at\t: {lastTimestamp:yyyy-MM-dd HH:mm:ss}");
Console.WriteLine($"Total trades retrieved\t: {trades.Count:N0}");
Console.WriteLine($"Cumulated trade sizes\t: {totalSizes:N0}");
Console.WriteLine($"Average trade price\t: {avgPrice:F3}");
Console.WriteLine($"Average trade size\t: {avgSize:N0}");
Console.WriteLine($"Max trade price\t\t: {maxPrice:F3}");
Console.WriteLine($"Min trade price\t\t: {minPrice:F3}");
Console.WriteLine($"Max trade size\t\t: {maxSize:N0}");
Console.WriteLine($"Min trade size\t\t: {minSize:N0}");
Console.WriteLine($"Vwap\t\t\t: {vwap:F3}");

#### 4.3 Plot the interval trades
Display trade prices and sizes over the look back period with the package of your choice.<br>*`XPlot.Plotly`* is used as per open source display package.

In [ ]:
var pricesAndVolumes = new List<Scattergl>
{ 
    new Scattergl
    {
        name = "Tradeprice",
        mode = "lines",
        x = trades.Select(elt => elt.TimeStamp.ToDateTime()),
        y = trades.Select(elt => elt.Price)
    },
    new Scattergl
    {
        name = "Tradesize",
        x = trades.Select(elt => elt.TimeStamp.ToDateTime()),
        y = trades.Select(elt => elt.Size),
        yaxis = "y2"
    }
};


var layout = new Layout.Layout{ 
    title = $"Interval trades for {ticker}-{exchange} | Trades count: {totalTrades} | Toal traded volume: {totalSizes} ",
    yaxis = new Yaxis {
        title = "Price", 
        autorange = true,
        side = "left",
    },
    yaxis2 = new Yaxis {
        title = "Size", 
        autorange = true,
        side = "right",  
    },
};

var chart = Chart.Plot(pricesAndVolumes, layout);
chart.WithXTitle($"Time");
chart.Width = 1500;
chart.Height = 800;
display(chart);